In [155]:
import pandas as pd
import numpy as np
from patsy import dmatrix, dmatrices
%pylab inline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split

# ------- PREPROCESSING --------
pump_data = pd.read_csv('pump_training_data.csv')
pump_data['yr_recorded'] = pump_data['date_recorded'].apply(lambda x: x.split('-')[0])
test_data = pd.read_csv('pump_test_data.csv')
test_data['yr_recorded'] = test_data['date_recorded'].apply(lambda x: x.split('-')[0])
pump_status = pd.read_csv('pump_status_group.csv')


pump_data['district_code'] = pump_data['district_code'].apply(str)
test_data['district_code'] = test_data['district_code'].apply(str)


pump_data.loc[:, 'yr_recorded'] = pump_data.loc[:, 'yr_recorded'].convert_objects(convert_numeric=True)
pump_data['age'] = pump_data['yr_recorded'] - pump_data['construction_year']
test_data.loc[:, 'yr_recorded'] = test_data.loc[:, 'yr_recorded'].convert_objects(convert_numeric=True)
test_data['age'] = test_data['yr_recorded'] - test_data['construction_year']



# introduce more information about geography
pump_data['long*lat'] = pump_data['longitude'] * pump_data['latitude']
pump_data['long-lat'] = pump_data['longitude'] - pump_data['latitude']
pump_data['long+lat'] = pump_data['longitude'] + pump_data['latitude']
pump_data['long*gps'] = pump_data['longitude'] * pump_data['gps_height']
pump_data['lat*gps'] = pump_data['latitude'] * pump_data['gps_height']
pump_data['long*lat*gps'] = pump_data['long*lat'] * pump_data['gps_height']
pump_data['long^2'] = pump_data['longitude'] * pump_data['longitude']
pump_data['lat^2'] = pump_data['latitude'] * pump_data['latitude']

test_data['long*lat'] = test_data['longitude'] * test_data['latitude']
test_data['long-lat'] = test_data['longitude'] - test_data['latitude']
test_data['long+lat'] = test_data['longitude'] + test_data['latitude']
test_data['long*gps'] = test_data['longitude'] * test_data['gps_height']
test_data['lat*gps'] = test_data['latitude'] * test_data['gps_height']
test_data['long*lat*gps'] = test_data['long*lat'] * test_data['gps_height']
test_data['long^2'] = test_data['longitude'] * test_data['longitude']
test_data['lat^2'] = test_data['latitude'] * test_data['latitude']


pumps = pd.merge(pump_data, pump_status, on='id')


pumps = pumps.drop(['date_recorded', 'id'], axis=1)

test_data = test_data.drop(['date_recorded', 'id'], axis=1)

# BIN LONGITUDE, LATITUDE, AMOUT_TSH, GPS_HEIGHT




Populating the interactive namespace from numpy and matplotlib


In [156]:


scrub_dict = {"world_vision": ["world vision/ kkkt", "world vision", "world vision/adra",
                               "world vision/rc church", "worldvision", "wvt", "vwc", "vwt",
                               "vw", "wvc", "world division", "world vission", "world visiin",
                               "word divisio"],
             "wedeco": ["wdeco", "we", "wedeco/wessons", "wedeko"],
             "water aid": ["water aid", "wateraid", "water hu","water users group",
                           "water user group", "water use group", "water solution",
                           "wate aid"],
             "ministry_water": ["ministry of watre", "ministry of water engineer",
                                "ministryof water"],
             "sema": ["sema", "water /sema", "water aid/sema", "water aid /sema",
                      "water /sema", "sema", "sema co ltd", "sema consultant",
                      "wate aid/sema", "sema s", "semaki", "semaki k"],
             "wananchi": ["wanan", "wananchi", "wananchi technicians"],
             "village": ["villagers", "villager", "villagerd", "villages", "villege council",
                         "villi", "villigers", "village water committee",
                         "village water commission", "village water attendant",
                         "village technician", "village office", "village local contractor",
                         "village govt", "village government", "village counil",
                         "village council orpha", "village council .oda", "village council",
                         "village community members", "village community", "village  council",
                         "village", "villag", "villaers", "villa", "vill"],
             "unicef": ["unicef", "unicrf", "unisef", "unice", "unice/ cspd", "unicef/ csp",
                        "unicef/african muslim agency","unicef/central", "unicef/cspd",
                        "uniceg", "unicet", "unicrf", "uniseg"],
             "twesa": ["twesa", "twe", "twesa /community", "twesa/ community",
                       "twesa/community", "twess", "tuwasa", "twe"],
             "tcrs": ["tcrs", "tcrs /care", "tcrs /dwe", "tcrs /government", "tcrs /twesa",
                      "tcrs a", "tcrs kibondo", "tcrs twesa", "tcrs/care", "tcrs/dwe",
                      "tcrs/tlc", "tcrs/twesa", "tcrs/village community",
                      "tcrs /government", "tcrs kibondo", "tcrst", "tag", "tag church",
                      "tag church ub"],
             "tasaf": ["tasa", "tasaf", "tasaf 1", "tasaf and community", "tasaf and lga",
                       "tasaf and mmem", "tasaf/", "tasaf/dmdd", "tasaf/tlc",
                       "tasafcitizen and lga", "tasf", "tassaf", "tassaf /tcrs",
                       "tassaf/ tcrs", "tassaf/tcrs", "government /tassaf",
                       "government/tassaf", "tasad", "tasae", "tasaf ii", "tasafu", "tasef",
                       "tassaf i", "tassaf ii", "tassaf/ danida"],
             "government": ["centr", "cental government", "central government", "central govt",
                            "tanzania goverment", "tanzania", "tanzanian government", "tanza",
                            "tanz", "gove", "gover", "goverm", "govern", "governme",
                            "governmen", "government", "government /community",
                            "government /sda", "government /tcrs", "government /world vision",
                            "government and community", "government of misri",
                            "government/tcrs", "tanza", "plan tanzania", "tanzania"],
             "shipo": ["ship", "shipo", "shipo constructors"],
             "rwssp": ["rwsp", "rwssp", "rural", "rural drinking water supply",
                       "rural drunkung water supply", "rural water supply", "national rural",
                       "rwsso", "rwssp shinyanga", "rwssp/wsdp", "rwsssp", 
                       "rural water supply and sanita", "rural water supply and sanitat"],
             "rwe": ["rwe", "rwe /community", "rwe community","rwe/ community",
                     "rwe/community", "rwe/dwe", "rwe/tcrs", "rwedwe", "rwet/wesa",
                     "region water department", "regional water",
                     "regional water engineer arusha", "regwa", "regwa company",
                     "region group", "region water"],
             "roman_catholic": ["roma", "r.c", "romam", "roman", "roman ca", "roman catholic",
                                "roman catholic rulenge diocese", "roman cathoric -kilomeni",
                                "roman cathoric -same", "roman cathoric same", "roman church",
                                "rc", "rc .church", "rc c", "rc churc", "rc church",
                                "rc church brother", "rc church/cefa",
                                "rc church/central gover", "rc mi", "rc mis", "rc mission",
                                "rc missionary", "rc msufi", "rc njoro", "rc/mission",
                                "rcchurch/cefa", "cathoric", "romam catholic",
                                "roman cathoric", "roman cathoric-same",
                                "roman cathoric church"],
             "private": ["priv", "priva", "privat", "private", "private company",
                         "private individuals", "private institutions", "private owned",
                         "private person", "private technician", "private co",
                         "private individual", "private individul"],
             "oxfarm": ["ox", "oxfam", "oxfarm", "oxfam gb", "oxfarm gb"],
             "norad": ["no", "nora", "norad"],
             "mwe": ["mwe", "mwe &"],
             "muwsa": ["muwasa", "muwaza", "muwsa", "mtuwasa", "mutwasa and community"],
             "missionary": ["missi", "missio", "mission", "missionaries", "missionary",
                           "missi", "missio", "mission", "missionaries", "missionary",
                           "church", "churc","christian outrich", "christan outrich"],
             "local": ["local", "local technician", "local local contract", "local fundi",
                       "local l technician", "local te", "local technical",
                       "local technical tec", "local technician", "local technitian",
                       "locall technician", "localtechnician"],
             "kuwait": ["kuwait", "kuwaiti", "kuweit"],
             "kkkt": ["kkkt", "kkkt-dioces ya pare", "kkkt _ konde and dwe", "kkkt canal",
                      "kkkt church", "kkkt dme", "kkkt katiti ju", "kkkt kilinga",
                      "kkkt leguruki", "kkkt mareu", "kkkt ndrumangeni", "kkt"],
             "kiliwater": ["kili water", "kiliwater", "kiliwater r"],
             "jica": ["jica", "jicks", "jika", "jaica", "jaica co"],
             "isf": ["is", "isf", "isf / tasaff", "isf and tacare", "isf/government",
                     "isf/tacare"],
             "idara": ["idara", "idara ya maji"],
             "hesawa": ["hesaw", "hesawa", "hesawq", "hesaws", "hesawz", "hesewa"],
             "fin_water": ["fw", "finwater", "finwate", "finw", "finn water", "fini water",
                           "fin water", "fiwater"],
             "dwe": ["consultant and dwe", "dw", "dw e", "dw#", "dw$", "dwe", "dwe & lwi",
                     "dwe /tassaf", "dwe&", "dwe/", "dwe/anglican church", "dwe/tassaf",
                     "dwe/ubalozi wa marekani", "dwe{", "dweb", "dwr", "dwe and veo",
                     "dwe/norad", "dwe/rudep", "dwe/ubalozi wa marekani", "water aid/dwe"],
             "district council": ["district council", "district counci", "district  council",
                                  "distri"],
             "water_dept": ["district water department", "district water depar",
                            "water department", "water authority", "water board",
                            "water boards", "rural water department"],
             "dh": ["dh", "dhv"],
             "danida": ["danida", "daniad", "danda", "danid", "danida co", "danids", "dannida",
                        "denish", "da"],
             "community": ["commu", "communit", "community"],
             "consultant_ces": ["ces", "consultant", "consultant engineer",
                                "consulting engineer", "consuting engineer", "cons"],
             "adra": ["adra", "adra /community", "adra /government", "adra/ community",
                      "adra/community", "adra/government", "0"],
             "acra": ["accra", "acra"],
             "plan_international": ["plan int", "plan", "plan internatio",
                                    "plan international"],
             "netherlands": ["nerthlands", "nethalan", "nethe", "netherla", "netherland"],
             "cefa": ["cefa", "cefa-njombe", "cefa/rcchurch"],
             "concern": ["conce", "concern", "concern /govern", "concern world wide",
                         "concern/governm", "co", "cocen", "cocern"],
             "ces": ["ces(gmbh)", "ces (gmbh)"],
             "germany": ["germany", "germany cristians", "germany misionary",
                         "germany missionary", "germany republi"],
             "unknown": ["", " ", "-", "not known", "unknown installer", "other"]}

# for x in [pumps, test_data]:
#     mask = x['installer'] in name_list


for key1, value1 in scrub_dict.iteritems():
    for key2, value2 in scrub_dict.iteritems():
        if key1!=key2:
            output = [val for val in value1 if val in value2]
            if len(output) > 0:
                print 'match in', key1+',', key2
                print output
                print

In [157]:
print test_data.shape
print pumps.shape

(14850, 48)
(59400, 49)


In [158]:
# pumps.loc[pumps['permit'].isnull(), 'permit'] = 'Unknown'
# pumps.loc[pumps['public_meeting'].isnull(), 'public_meeting'] = 'Unknown'
# pumps.loc[pumps['scheme_management'].isnull(), 'scheme_management'] = 'Other'
# test_data.loc[test_data['permit'].isnull(), 'permit'] = 'Unknown'
# test_data.loc[test_data['public_meeting'].isnull(), 'public_meeting'] = 'Unknown'
# test_data.loc[test_data['scheme_management'].isnull(), 'scheme_management'] = 'Other'







In [159]:
def pre_process(df):
    for col in df.columns.values:
        if type(df.loc[0, col])==type(''):
            df.loc[df[col].isnull(), col] = 'unknown'
            df[col] = df[col].str.lower()
            for key, str_list in scrub_dict.iteritems():
                for val in str_list:
                    mask = (df[col]==val)
                    df.loc[mask, col] = key
            # remove any categories not appearing at least 100 times
            for z in df[col].unique():
                if len(df[df[col]==z]) < 500:
                    df.loc[df[col]==z, col] = 'unknown'
        else:
            df.loc[df[col].isnull(), col] = 0
    return df

def pre_process2(df):
    for col in df.columns.values:
        if type(df.loc[0, col])==type(''):
            df.loc[df[col].isnull(), col] = 'unknown'
            df[col] = df[col].str.lower()
            for key, str_list in scrub_dict.iteritems():
                for val in str_list:
                    mask = (df[col]==val)
                    df.loc[mask, col] = key
            # remove any categories not appearing at least 100 times
            for z in df[col].unique():
                if len(df[df[col]==z]) < 100:
                    df.loc[df[col]==z, col] = 'unknown'
        else:
            df.loc[df[col].isnull(), col] = 0
    return df

            
            
pumps = pre_process2(pumps)
test_data = pre_process(test_data)

print pumps.isnull().sum().sum()
print test_data.isnull().sum().sum()

0
0


In [160]:


# if len(whatever.unique()) > 200? ...
    # then convert all not in top 200 to 'unknown'


In [161]:
for z in pumps['payment_type'].unique():
    where_z = pumps[pumps['payment_type']==z]
    print z, len(pumps[pumps['payment_type']==z])

annually 3642
never pay 25348
per bucket 8985
unknown 9211
on failure 3914
monthly 8300


In [162]:
formula = 'status_group ~ ' + ' + '.join(['Q("'+x+'")' for x in pumps.columns.values[:-1]])
formula

'status_group ~ Q("amount_tsh") + Q("funder") + Q("gps_height") + Q("installer") + Q("longitude") + Q("latitude") + Q("wpt_name") + Q("num_private") + Q("basin") + Q("subvillage") + Q("region") + Q("region_code") + Q("district_code") + Q("lga") + Q("ward") + Q("population") + Q("public_meeting") + Q("recorded_by") + Q("scheme_management") + Q("scheme_name") + Q("permit") + Q("construction_year") + Q("extraction_type") + Q("extraction_type_group") + Q("extraction_type_class") + Q("management") + Q("management_group") + Q("payment") + Q("payment_type") + Q("water_quality") + Q("quality_group") + Q("quantity") + Q("quantity_group") + Q("source") + Q("source_type") + Q("source_class") + Q("waterpoint_type") + Q("waterpoint_type_group") + Q("yr_recorded") + Q("age") + Q("long*lat") + Q("long-lat") + Q("long+lat") + Q("long*gps") + Q("lat*gps") + Q("long*lat*gps") + Q("long^2") + Q("lat^2")'

In [163]:
Y, X = dmatrices(formula, pumps, return_type='dataframe')
X.shape

(59400, 541)

In [164]:
test_formula = ' + '.join(['Q("'+x+'")' for x in test_data.columns.values[:-1]])
actual_test = dmatrix(test_formula, test_data, return_type='dataframe')
actual_test.shape

(14850, 1912)

In [165]:
# remove columns from train and test that are not found in the other
x_not_actual = ['Intercept']
actual_not_x = ['Intercept']

for i in actual_test.columns.values:
    if i not in X.columns.values:
        actual_not_x.append(i)

for i in X.columns.values:
    if i not in actual_test.columns.values:
        x_not_actual.append(i)

X = X.drop(x_not_actual, axis=1)
actual_test = actual_test.drop(actual_not_x, axis=1)

print X.shape
print actual_test.shape

(59400, 129)
(14850, 129)


In [166]:
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

model_rf = RandomForestClassifier(n_estimators=50, random_state=78, n_jobs=-1)
model_rf.fit(X_train, y_train)

prediction = model_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, prediction)
accuracy

0.78226711560044893

In [167]:
rf_pred = pd.DataFrame(prediction, columns=['functional', 'needs_repair', 'non_functional'])
y_true = pd.DataFrame(y_test, columns=['functional', 'needs_repair', 'non_functional'])

#pd.crosstab(rf_pred.index, y_true.columns, row_names=rf_pred.columns.values)
# pumps.groupby(['functional'])
y_true['status_group'] = 'functional'
y_true.loc[y_true['needs_repair']==1, 'status_group'] = 'needs_repair'
y_true.loc[y_true['non_functional']==1, 'status_group'] = 'non_functional'


rf_pred['status_group'] = 'functional'
rf_pred.loc[rf_pred['needs_repair']==1, 'status_group'] = 'needs_repair'
rf_pred.loc[rf_pred['non_functional']==1, 'status_group'] = 'non_functional'

rf_pred['true_status'] = y_true['status_group']


xtab = pd.crosstab(rf_pred['true_status'], rf_pred['status_group'], margins=True)
xtab

status_group,functional,needs_repair,non_functional,All
true_status,,,,
functional,8804,192,695,9691
needs_repair,756,355,159,1270
non_functional,1532,83,5244,6859
All,11092,630,6098,17820


In [168]:
def convert_pred_to_df(np_array):
    pred_df = pd.DataFrame(pd.read_csv('pump_test_data.csv')['id'])
    pred_df['status_group'] = 'none'
    for i, ans in enumerate(np_array):
        if ans[0]==1:
            pred_df.loc[i, 'status_group'] = 'functional'
        elif ans[1]==1:
            pred_df.loc[i, 'status_group'] = 'functional needs repair'
        else:
            pred_df.loc[i, 'status_group'] = 'non functional'
    return pred_df

In [169]:
model_rf_test = RandomForestClassifier(n_estimators=50, random_state=78, n_jobs=-1)
model_rf_test.fit(X, Y)

actual_pred = model_rf_test.predict(actual_test)
submission = convert_pred_to_df(actual_pred)

submission.to_csv('submission_unclean.csv', index=False)

In [170]:
submission[:20]

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
5,52449,functional
6,24806,functional
7,28965,non functional
8,36301,non functional
9,54122,functional


In [171]:
submission.loc[submission['status_group']=='none', :]

,id,status_group


In [172]:
submission.shape

(14850, 2)

In [173]:
X[:20]

,"Q(""funder"")[T.government of tanzania]","Q(""funder"")[T.hesawa]","Q(""funder"")[T.unknown]","Q(""installer"")[T.dwe]","Q(""installer"")[T.government]","Q(""installer"")[T.unknown]","Q(""wpt_name"")[T.unknown]","Q(""basin"")[T.lake nyasa]","Q(""basin"")[T.lake rukwa]","Q(""basin"")[T.lake tanganyika]",...,"Q(""construction_year"")","Q(""yr_recorded"")","Q(""age"")","Q(""long*lat"")","Q(""long-lat"")","Q(""long+lat"")","Q(""long*gps"")","Q(""lat*gps"")","Q(""long*lat*gps"")","Q(""long^2"")"
0,0,0,0,0,0,0,0,1,0,0,...,1999,2011,12,-344.361084,44.794415,25.081771,48563.948923,-13700.287260,-478661.907002,1220.670325
1,0,0,1,0,0,1,0,0,0,0,...,2010,2013,3,-74.514410,36.846232,32.551300,48543.573774,-3004.304500,-104245.659149,1204.004369
2,0,0,1,0,0,0,1,0,0,0,...,2009,2013,4,-143.149506,41.281993,33.639336,25698.015820,-2621.431372,-98200.561016,1403.301382
3,0,0,0,0,0,0,1,0,0,0,...,1986,2013,27,-429.324583,49.641459,27.330863,10121.860311,-2933.843300,-112912.365254,1481.184579
4,0,0,1,0,0,0,0,0,0,0,...,0,2011,2011,-56.824967,32.956206,29.305488,0.000000,-0.000000,-0.000000,969.129617
5,0,0,0,1,0,0,1,0,0,0,...,2009,2011,2,-186.681376,43.938383,34.407208,0.000000,-0.000000,-0.000000,1534.507915
6,0,0,0,0,0,0,1,0,0,0,...,0,2012,2012,-125.655003,37.128775,29.596045,0.000000,-0.000000,-0.000000,1113.050389
7,0,0,0,1,0,0,1,0,0,1,...,0,2012,2012,-137.861187,36.846815,28.394419,0.000000,-0.000000,-0.000000,1064.104658
8,0,0,0,0,0,0,1,0,0,1,...,0,2012,2012,-168.354605,37.857812,27.564388,0.000000,-0.000000,-0.000000,1070.016064
9,0,0,1,0,0,0,1,0,0,0,...,0,2011,2011,-38.499677,31.884041,29.369940,0.000000,-0.000000,-0.000000,938.012549


In [174]:
print X['Q("gps_height")'].max()
print X['Q("gps_height")'].min()
print X['Q("longitude")'].max()
print X['Q("longitude")'].min()
print X['Q("latitude")'].max()
print X['Q("latitude")'].min()
print X['Q("amount_tsh")'].max()
print X['Q("amount_tsh")'].min()

2770.0
-90.0
40.34519307
0.0
-2e-08
-11.64944018
350000.0
0.0


In [175]:
pd.DataFrame(model_rf_test.feature_importances_, index=X.columns).sort(0, ascending=False)

,0
"Q(""long+lat"")",0.061007
"Q(""long-lat"")",0.060974
"Q(""long*lat"")",0.059692
"Q(""long^2"")",0.059346
"Q(""longitude"")",0.059098
"Q(""latitude"")",0.058940
"Q(""lat*gps"")",0.030149
"Q(""long*gps"")",0.029817
"Q(""long*lat*gps"")",0.029519
"Q(""gps_height"")",0.029021
